In [12]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
import tensorflow as tf
import numpy as np
import os

In [13]:
import pandas as pd
from pathlib import Path
download_path = Path.cwd() / 'UrbanSound8K'

metadata_file = download_path / 'metadata' / 'test.csv'
df1 = pd.read_csv(metadata_file)
print(df1.head())

  slice_file_name                                        folder_name  classID  \
0          10.wav  C:\Users\sm251206\PycharmProjects\SC\UrbanSoun...        0   
1         100.wav  C:\Users\sm251206\PycharmProjects\SC\UrbanSoun...        0   
2          11.wav  C:\Users\sm251206\PycharmProjects\SC\UrbanSoun...        0   
3          12.wav  C:\Users\sm251206\PycharmProjects\SC\UrbanSoun...        0   
4          13.wav  C:\Users\sm251206\PycharmProjects\SC\UrbanSoun...        0   

  class_name  
0  Amir_Levy  
1  Amir_Levy  
2  Amir_Levy  
3  Amir_Levy  
4  Amir_Levy  


In [14]:
# import os
# download_path = Path.cwd() / 'UrbanSound8K'/'audio'/'fold11'
# arr = os.listdir(download_path)
# arr

In [10]:
oudio_path = Path.cwd() / 'UrbanSound8K' / 'audio'
for folder_path1, d, f1 in os.walk(oudio_path):
    print(folder_path1)


C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio
C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio\Amir_Levy
C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio\Elad_Peleg
C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio\gal tzhaik
C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio\Gidi_Aviezer
C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio\omri
C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio\Ronit_levin
C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio\rotem
C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio\shlomo ben ami


In [20]:
import csv   
import os
from pathlib import Path

csv_path = Path.cwd() / 'UrbanSound8K' / 'metadata' / 'test.csv'
oudio_path = Path.cwd() / 'UrbanSound8K' / 'audio'
colms = ["slice_file_name" , "folder_name" , "classID" , "class_name"]
folders = os.listdir(oudio_path)[1:]
with open(csv_path, "w") as f:
    writer = csv.writer(f)
    writer.writerow(colms)
    counter = -1
    for folder_path, d, f in os.walk(oudio_path):
        print("folder_path",folder_path)
        for oudio_file in f[1:]:
            print(oudio_file)
            if counter < len(folders):
#                     os.rename(f'{folder_path}\\{oudio_file}',f'{folder_path}\\{oudio_file.replace(".mp3","")}')
                x = [oudio_file , folder_path , counter , folders[counter]]
                print(counter)
                writer.writerow(x)
        counter += 1


folder_path C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio
folder_path C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio\Amir_Levy
10.wav
0
100.wav
0
11.wav
0
12.wav
0
13.wav
0
14.wav
0
15.wav
0
16.wav
0
17.wav
0
18.wav
0
19.wav
0
2.wav
0
20.wav
0
21.wav
0
22.wav
0
23.wav
0
24.wav
0
25.wav
0
26.wav
0
27.wav
0
28.wav
0
29.wav
0
3.wav
0
30.wav
0
31.wav
0
32.wav
0
33.wav
0
34.wav
0
35.wav
0
36.wav
0
37.wav
0
38.wav
0
39.wav
0
4.wav
0
40.wav
0
41.wav
0
42.wav
0
43.wav
0
44.wav
0
45.wav
0
46.wav
0
47.wav
0
48.wav
0
49.wav
0
5.wav
0
50.wav
0
51.wav
0
52.wav
0
53.wav
0
54.wav
0
55.wav
0
56.wav
0
57.wav
0
58.wav
0
59.wav
0
6.wav
0
60.wav
0
61.wav
0
62.wav
0
63.wav
0
64.wav
0
65.wav
0
66.wav
0
67.wav
0
68.wav
0
69.wav
0
7.wav
0
70.wav
0
71.wav
0
72.wav
0
73.wav
0
74.wav
0
75.wav
0
76.wav
0
77.wav
0
78.wav
0
79.wav
0
8.wav
0
80.wav
0
81.wav
0
82.wav
0
83.wav
0
84.wav
0
85.wav
0
86.wav
0
87.wav
0
88.wav
0
89.wav
0
9.wav
0
90.wav
0
91.wav
0
92.wav
0
93.wav
0
94.wav
0
95.wav
0
96.wav

In [15]:
from pathlib import Path

os.listdir(oudio_path)

NameError: name 'oudio_path' is not defined

In [16]:

# df1['relative_path'] = '\\fold' + df1['folder_name'].astype(str) + '\\' + df1['slice_file_name'].astype(str)
df1['relative_path'] = df1['folder_name'].astype(str) + '\\' + df1['slice_file_name'].astype(str)
df = df1[['relative_path', 'classID']]

# df1['relative_path'] = '\\fold' + df1['fold'].astype(str) + '\\' + df1['slice_file_name'].astype(str)
# df = df1[['relative_path', 'class']]
print(df.head())
print(df1["relative_path"][0])

                                       relative_path  classID
0  C:\Users\sm251206\PycharmProjects\SC\UrbanSoun...        0
1  C:\Users\sm251206\PycharmProjects\SC\UrbanSoun...        0
2  C:\Users\sm251206\PycharmProjects\SC\UrbanSoun...        0
3  C:\Users\sm251206\PycharmProjects\SC\UrbanSoun...        0
4  C:\Users\sm251206\PycharmProjects\SC\UrbanSoun...        0
C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio\Amir_Levy\10.wav


In [17]:
# Load an audio file. Return the signal as a tensor and the sample rate
# ----------------------------
class AudioUtil():
    @staticmethod
    def open(audio_file):
        sig, sr = torchaudio.load(audio_file)
        return (sig, sr)

  # ----------------------------
  # Convert the given audio to the desired number of channels
  # ----------------------------
    @staticmethod
    def rechannel(aud, new_channel):
        sig, sr = aud

        if (sig.shape[0] == new_channel):
            # Nothing to do
            return aud

        if (new_channel == 1):
            # Convert from stereo to mono by selecting only the first channel
            resig = sig[:1, :]
        else:
            # Convert from mono to stereo by duplicating the first channel
            resig = torch.cat([sig, sig])

        return ((resig, sr))

  # ----------------------------
  # Since Resample applies to a single channel, we resample one channel at a time
  # ----------------------------
    @staticmethod
    def resample(aud, newsr):
        sig, sr = aud

        if (sr == newsr):
            # Nothing to do
            return aud

        num_channels = sig.shape[0]
        # Resample first channel
        resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
        if (num_channels > 1):
            # Resample the second channel and merge both channels
            retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
            resig = torch.cat([resig, retwo])

        return ((resig, newsr))

  # ----------------------------
  # Pad (or truncate) the signal to a fixed length 'max_ms' in milliseconds
  # ----------------------------
    @staticmethod
    def pad_trunc(aud, max_ms):
        sig, sr = aud
        num_rows, sig_len = sig.shape
        max_len = sr//1000 * max_ms

        if (sig_len > max_len):
            # Truncate the signal to the given length
            sig = sig[:,:max_len]

        elif (sig_len < max_len):
            # Length of padding to add at the beginning and end of the signal
            pad_begin_len = random.randint(0, max_len - sig_len)
            pad_end_len = max_len - sig_len - pad_begin_len

            # Pad with 0s
            pad_begin = torch.zeros((num_rows, pad_begin_len))
            pad_end = torch.zeros((num_rows, pad_end_len))

            sig = torch.cat((pad_begin, sig, pad_end), 1)

        return (sig, sr)

  # ----------------------------
  # Shifts the signal to the left or right by some percent. Values at the end
  # are 'wrapped around' to the start of the transformed signal.
  # ----------------------------
    @staticmethod
    def time_shift(aud, shift_limit):
        sig,sr = aud
        _, sig_len = sig.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return (sig.roll(shift_amt), sr)
  # ----------------------------
  # Generate a Spectrogram
  # ----------------------------
    @staticmethod
    def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
        sig,sr = aud
        top_db = 80

        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

        # Convert to decibels
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        return (spec)
  # ----------------------------
  # Augment the Spectrogram by masking out some sections of it in both the frequency
  # dimension (ie. horizontal bars) and the time dimension (vertical bars) to prevent
  # overfitting and to help the model generalise better. The masked sections are
  # replaced with the mean value.
  # ----------------------------
    @staticmethod
    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        _, n_mels, n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec

        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
            aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

        time_mask_param = max_mask_pct * n_steps
        for _ in range(n_time_masks):
            aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

        return aug_spec

In [18]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio

# ----------------------------
# Sound Dataset
# ----------------------------
class SoundDS(Dataset):
    def __init__(self, df, data_path):
        self.df = df
        self.data_path = str(data_path)
        self.duration = 4000
        self.sr = 44100
        self.channel = 2
        self.shift_pct = 0.4
            
  # ----------------------------
  # Number of items in dataset
  # ----------------------------
    def __len__(self):
        return len(self.df)    
    
  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
    def __getitem__(self, idx):
        # Absolute file path of the audio file - concatenate the audio directory with
        # the relative path
#         print(self.data_path)
#         print(self.df.loc[idx, 'relative_path'])
#         audio_file = self.data_path + self.df.loc[idx, 'relative_path']
        audio_file = self.df.loc[idx, 'relative_path']
        # Get the Class ID
        class_id = self.df.loc[idx, 'classID']
#         print(audio_file)
        aud = AudioUtil.open(audio_file)
        # Some sounds have a higher sample rate, or fewer channels compared to the
        # majority. So make all sounds have the same number of channels and same 
        # sample rate. Unless the sample rate is the same, the pad_trunc will still
        # result in arrays of different lengths, even though the sound duration is
        # the same.
        reaud = AudioUtil.resample(aud, self.sr)
        rechan = AudioUtil.rechannel(reaud, self.channel)

        dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
        shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
        sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
        aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

        return aug_sgram, class_id

In [19]:
from torch.utils.data import random_split

myds = SoundDS(df1, r"C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio")
# myds = SoundDS(df1, r"C:\Users\sm251206\PycharmProjects\SC\UrbanSound8K\audio")
# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

In [25]:
import torch.nn.functional as F
from torch.nn import init
import torch.nn as nn

# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Second Convolution Block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Second Convolution Block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]
        
        self.conv5 = nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu5 = nn.ReLU()
        self.bn5 = nn.BatchNorm2d(128)
        init.kaiming_normal_(self.conv5.weight, a=0.1)
        self.conv5.bias.data.zero_()
        conv_layers += [self.conv5, self.relu5, self.bn5]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=128, out_features=8)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)

        # Final output
        return x

# Create the model and put it on the GPU if available
myModel = AudioClassifier()
devMice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device

device(type='cpu')

In [ ]:
# ----------------------------
# Training Loop
# ----------------------------
epochs = []
Losses = []
Accuracy = []
def training(model, train_dl, num_epochs):
    # Loss Function, Optimizer and Scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

    # Repeat for each epoch
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0

        # Repeat for each batch in the training set
        for i, data in enumerate(train_dl):
#             print(data[0].to(device))
#             print(data[1].to(device))
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Keep stats for Loss and Accuracy
            running_loss += loss.item()

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

            #if i % 10 == 0:    # print every 10 mini-batches
            #    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))

        # Print stats at the end of the epoch
        num_batches = len(train_dl)
        avg_loss = running_loss / num_batches
        acc = correct_prediction/total_prediction
        epochs.append(epoch)
        Losses.append(avg_loss)
        Accuracy.append(acc)
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

    print('Finished Training')
  
num_epochs = 15  # Just for demo, adjust this higher.
training(myModel, train_dl, num_epochs)

Epoch: 0, Loss: 2.00, Accuracy: 0.27
Epoch: 1, Loss: 1.66, Accuracy: 0.58
Epoch: 2, Loss: 1.20, Accuracy: 0.73
Epoch: 3, Loss: 0.75, Accuracy: 0.86
Epoch: 4, Loss: 0.47, Accuracy: 0.93
Epoch: 5, Loss: 0.34, Accuracy: 0.93
Epoch: 6, Loss: 0.27, Accuracy: 0.96
Epoch: 7, Loss: 0.25, Accuracy: 0.95


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(epochs, Losses)
plt.xlabel('epochs')
plt.ylabel('Loss')

In [24]:
,# ----------------------------
# Inference
# ----------------------------
def inference (model, val_dl):
    correct_prediction = 0
    total_prediction = 0
    calsses = df1["class"].unique()
    
  # Disable gradient updates
    with torch.no_grad():
        for data in val_dl:
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)
#             print("labels",labels)
            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
#             print("inputs",inputs)
            outputs = model(inputs)
            arr = []
            arr2 = []
#             for tensor in outputs:
#                 arr.append(torch.argmax(tensor))
#             print(arr)
            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
#             print(prediction)
            for pred in prediction:
                arr.append(calsses[pred])
            for clas in labels:            
                arr2.append(calsses[clas])
            x={"lables":arr2,"prediction":arr,"could_predict":np.array(arr2)==np.array(arr)}
            y = pd.DataFrame(x,columns = ["lables","prediction","could_predict"])
#             print(y)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]
    
    acc = correct_prediction/total_prediction
    print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

# Run inference on trained model with the validation set
inference(myModel, val_dl)

KeyError: 'class'

In [8]:
data1 = [[ 0.8478, -2.1911,  0.3530,  0.5796,  3.5831, -0.3604, -1.0573, -0.0778,
         -1.1122, -1.5201],
        [ 2.9137, -3.4620, -0.6290, -1.4556, -3.1236,  5.5903, -0.5580, -1.3530,
          0.2575,  0.7896],
        [-0.7440, -1.9484,  2.2594,  0.5511,  0.6669, -3.1220, -3.3023, -2.4344,
          3.2727,  1.4789],
        [ 0.6042, -2.5187,  0.4943, -0.1168,  0.5690,  2.2344, -1.3129,  0.2835,
         -0.5683, -0.4194],
        [-1.9126,  1.4242, -1.1883,  2.3374,  0.8311, -1.8455,  1.4622,  1.5770,
         -0.7302, -2.3551],
        [ 0.6497, -3.1684,  0.8624, -2.8300, -0.8037,  3.6699, -3.0725,  2.8194,
          0.1894,  0.9282],
        [-1.1475, -2.3690,  2.8642,  1.1853, -2.1871, -0.2715, -2.9640, -0.0746,
          0.4833,  3.7463],
        [-0.6595, -2.2410,  3.0515,  1.6381, -3.9516,  0.7598, -2.1772, -0.7236,
          0.5163,  2.9939],
        [ 1.7798, -2.1815, -1.2146, -3.2770,  2.8047,  1.7006, -1.7545,  4.3299,
         -1.8615, -0.2080],
        [ 1.3447, -0.1632, -0.5862,  0.7161,  0.4992, -0.2344,  0.7574,  0.0107,
         -2.7560,  1.0245],
        [ 0.3325, -0.4720, -0.6025, -2.9307,  4.2418, -1.2372, -2.9686,  1.6706,
          1.5448, -0.3943],
        [ 1.6692, -1.0705,  1.8209,  2.0996, -0.4041, -0.2788, -1.3297, -3.1263,
          0.3313, -0.5263],
        [ 0.4879, -3.4047,  2.3624,  0.8100, -0.9141,  0.7684, -1.3902, -1.3669,
          0.2264,  0.7574],
        [-1.1710,  1.5680, -1.9233,  1.5371,  0.2840, -1.3424,  2.7950,  0.4718,
         -0.7152, -1.6051],
        [ 1.5455, -1.5640,  0.7353, -0.4752, -1.1227, -0.5960, -2.2975, -1.6654,
          2.2399,  1.4472],
        [-1.1416,  5.9047, -1.8216, -1.1541,  1.3022, -2.6710, -0.6799, -1.0217,
          2.1397, -0.3293]]